In [1]:
import tensorflow as tf

2023-01-29 17:52:15.772347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 17:52:16.010778: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-29 17:52:16.061831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-01-29 17:52:16.061848: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above c

In [7]:
tf.random.normal(shape=(1, 2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.81158537, -0.7471892 ]], dtype=float32)>

In [38]:
tf.random.stateless_binomial(shape=(1,1), seed = [10, 1], counts = [5], probs = [0.4])

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[3]], dtype=int32)>

In [ ]:
tf.random.